In [ ]:
"""
Created on Mon Oct 21 16:13 2024

Prepare what's needed for eORCA1 on the ice-shelf side

@author: Clara Burgard
"""

In [139]:
import xarray as xr
import numpy as np
from cdo import Cdo
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from datetime import date

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 2.0.3


In [4]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

REMAP THE ICE SHELF CONCENTRATION

In [136]:
fNEMO_mask = 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'
dom_cfg_path = inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'
mask_file_path = inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc'
grid_file_path = inputpath_interim + 'grid_eORCA1_T.nc'

In [ ]:
cdo.selvar('isf_conc', input = mask_file_path, output = inputpath_interim + 'isf_conc_only_2km.nc')

In [ ]:
cdo.remapbil(grid_file_path, input = inputpath_interim + 'isf_conc_only_2km.nc', output = inputpath_interim + 'isf_conc_only_targetgrid.nc')

REMAP THE IMBIE MASK

In [ ]:
cdo.selvar('Iceshelf_extrap', input = mask_file_path, output = inputpath_interim + 'isfmask_only_2km.nc')

In [ ]:
cdo.remapnn(grid_file_path, input = inputpath_interim + 'isfmask_only_2km.nc', output = inputpath_interim + 'isfmask_only_targetgrid.nc')

CREATE MASK_VARIABLE

In [7]:
domain_cfg = xr.open_dataset(dom_cfg_path)
isfmask = xr.open_dataset(inputpath_interim + 'isfmask_only_targetgrid.nc')
isfconc = xr.open_dataset(inputpath_interim + 'isf_conc_only_targetgrid.nc')

In [148]:
ds_isfNEMO = xr.Dataset()

ds_isfNEMO['mask_isf'] = isfmask['Iceshelf_extrap'].where(domain_cfg['isf_draft'] > 0)
ds_isfNEMO['mask_isf'].attrs['standard_name'] = 'mask of all the ice shelves'
ds_isfNEMO['mask_isf'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

#ds_isfNEMO['mask_isf_onlyisfconc1'] = ds_isfNEMO['mask_isf'].where(isfconc['isf_conc'] > 0.9999)


In [9]:
ds_isfNEMO['floating_frac'] = isfconc['isf_conc'].where(isfconc['isf_conc'] < 0.9999,1)

CREATE MASK_ISF_OPEN AND MASK_ISF_CLOSED

In [10]:
file_Justine = xr.open_dataset(mask_file_path).drop('quantile')

In [11]:
for iid in file_Justine.ID:
    print(iid.values,file_Justine['NAME'].sel(ID=iid).values)

1 Jelbart
2 Fimbul
3 
4 Vigrid
5 Nivl
6 
7 Lazarev
8 Borchgrevink
9 Baudouin
10 
11 Prince_Harald
12 
13 Shirase
14 Hannan/Telen/Skallen
15 Rayner_Thyer
16 Zubchatyy/Porter/Myers
17 WilmaRobertDowner
18 Edward_VIII
19 
20 Utsikkar/Mulebreen/Cirque_Fjord/Hoseason/Rund_Bay
21 Amery
22 Publications
23 Sorsdal/Hovde/Flatnes/Dalk
24 West
25 Helen
26 Shackleton
27 Tracy_Tremenchus
28 
29 Conger_Glenzer
30 
31 Vincennes_Bay/Underwood
32 Williamson/Whittle/FoxE
33 Totten
34 Moscow_University
35 
36 Voyeykov
37 
38 Holmes
39 Frost
40 May_Glacier/Morse/Sandford
41 Dibble
42 WattBay/Zelee/Astrolabe/Liotard/Francais/Marret/Commandant_Charcot//PourquoiPas
43 Mertz
44 Ainsworth/Fisher
45 Ninnis
46 Deakin/HornBluff
47 Cook
48 Andreyev/Slava
49 Lauritzen
50 Drury
51 Matusevitch
52 Suvorov/Gillet/Noll/Paternostro/McLeod
53 Rennick
54 Chugunov/Barber/Gannutz
55 Lillie
56 Arneb/Manhaul/Ironside/Quatermain_Point/Moubray/Dennistoun/SmithInlet//Kirkby
57 Tucker
58 Mandible_Cirque
59 Mariner
60 Parker/Falkne

In [12]:
ID_open_list = [1,2,21,66,67,117,124,125,127,128]

In [13]:
file_Justine_open = file_Justine.sel(ID=ID_open_list)
file_Justine_closed = file_Justine.drop_sel(ID=ID_open_list)

In [14]:
domain_cfg_closed =  domain_cfg.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    for vv in ['bottom_level','top_level','isf_draft','bathy_metry']:
        domain_cfg_closed[vv] = domain_cfg_closed[vv].where(ds_isfNEMO['mask_isf'].drop_vars(['glamt', 'gphit']) != id_closed.values, 0)

  0%|          | 0/123 [00:00<?, ?it/s]

In [ ]:
ds_isfNEMO['mask_isf'].plot()

In [16]:
mask_isf_open = ds_isfNEMO['mask_isf'].copy()
for id_closed in tqdm(file_Justine_closed.ID):
    mask_isf_open = mask_isf_open.where(ds_isfNEMO['mask_isf'].drop_vars(['glamt', 'gphit']) != id_closed.values)

  0%|          | 0/123 [00:00<?, ?it/s]

In [17]:
mask_isf_closed = ds_isfNEMO['mask_isf'].where(np.isnan(mask_isf_open))

In [150]:
# create mask_isf_open and mask_isf_closed
ds_isfNEMO['mask_isf_open'] = mask_isf_open
ds_isfNEMO['mask_isf_open'].attrs['standard_name'] = 'mask of the ice shelves that are open'
ds_isfNEMO['mask_isf_open'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

ds_isfNEMO['mask_isf_closed'] = mask_isf_closed
ds_isfNEMO['mask_isf_closed'].attrs['standard_name'] = 'mask of the ice shelves that are closed'
ds_isfNEMO['mask_isf_closed'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

IDENTIFY FRONT

In [27]:
mask_0_1_2 = ds_isfNEMO['mask_isf'] * 2
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] != 0, 400) # land
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] == 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(domain_cfg['isf_draft'] == 0, 200) # ice shelf

In [71]:
# mask_open2 is equivalent to mask_isf_closed
mask_0_1_2_corr2 = mask_0_1_2.copy()
extrap_domains_closed = isfmask['Iceshelf_extrap'].copy()

for kisf in ID_open_list:
    mask_0_1_2_closed = mask_0_1_2_corr2.where(ds_isfNEMO['mask_isf'] != kisf, 400)
    extrap_domains_closed = extrap_domains_closed.where(extrap_domains_closed != kisf, np.nan)

In [59]:
is_mask0 = mask_isf_open.where(np.isnan(mask_isf_open)==False,0)
# set all ice shelves to 3
mask_front = mask_0_1_2_corr2.where((mask_0_1_2_corr2 == 0) | (mask_0_1_2_corr2 == 400), 300).copy()

In [109]:
mask_0_1_2_corr2.plot()

In [74]:
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
mask_front = mask_front.where((mask_front.shift(x=-1)-mask_front)!=300,500)
mask_front = mask_front.where((mask_front.shift(x=1)-mask_front)!=300,500)
mask_front = mask_front.where((mask_front.shift(y=-1)-mask_front)!=300,500)
mask_front = mask_front.where((mask_front.shift(y=1)-mask_front)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,extrap_domains_closed)

MAKE MAPS OF ZMIN AND ZMAX ON THE FRONT

In [106]:
zmin_front = mask_front.copy()
zmax_front = mask_front.copy()

for id_closed in tqdm(file_Justine_closed.ID):
    zmin_front = zmin_front.where(mask_front != id_closed, file_Justine_closed['z_min'].sel(ID=id_closed))
    zmax_front = zmax_front.where(mask_front != id_closed, file_Justine_closed['z_max'].sel(ID=id_closed))

  0%|          | 0/123 [00:00<?, ?it/s]

In [159]:
file_Justine_closed['z_min'].plot()
file_Justine_closed['z_perc01'].plot()

In [160]:
file_Justine_closed['z_max'].plot()
file_Justine_closed['z_perc99'].plot()

DISTRIBUTE MELT AT THE FRONT NORMALISED BY CELL SIZE

In [117]:
cell_area = domain_cfg['e1t'] * domain_cfg['e2t']
melt_src = 'Adusumilli' # options: 'Adusumilli','Rignot', 'Paolo', 'Davison'
melt_flux = file_Justine_closed['Melt'+melt_src]

In [120]:
melt_front = mask_front.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    cell_area_isf = cell_area.where(mask_front == id_closed)
    cell_area_isf_sum = cell_area.where(mask_front == id_closed).sum()
    melt_front = melt_front.where(mask_front != id_closed, melt_flux.sel(ID=id_closed) * cell_area_isf / cell_area_isf_sum)

  0%|          | 0/123 [00:00<?, ?it/s]

Double-check that the total melt is matched

In [122]:
melt_front_tot_list = []
for id_closed in tqdm(file_Justine_closed.ID):
    melt_front_tot_list.append(melt_front.where(mask_front == id_closed).sum().assign_coords({'ID': id_closed}))
melt_front_tot_xr = xr.concat(melt_front_tot_list, dim='ID')

  0%|          | 0/123 [00:00<?, ?it/s]

WRITE TO NETCDF

In [154]:
ds_isfNEMO['z_min'] = zmin_front
ds_isfNEMO['z_min'].attrs['standard_name'] = 'most shallow point of the ice-shelf draft indicated at the ice-shelf front'
ds_isfNEMO['z_min'].attrs['units'] = 'm below sea level'

ds_isfNEMO['z_max'] = zmax_front
ds_isfNEMO['z_max'].attrs['standard_name'] = 'deepest point of the ice-shelf draft indicated at the ice-shelf front'
ds_isfNEMO['z_max'].attrs['units'] = 'm below sea level'

ds_isfNEMO['melt_isf_closed'] = melt_front
ds_isfNEMO['melt_isf_closed'].attrs['standard_name'] = 'melt flux from '+melt_src+' distributed at the ice-shelf front'
ds_isfNEMO['melt_isf_closed'].attrs['units'] = 'Gt/yr'

ds_isfNEMO['ID'].attrs['standard_name'] = 'ice shelf ID'

In [155]:
ds_isfNEMO.attrs=dict(Source='Based on IMBIE2 (Mask_Iceshelf_IMBIE2_v2.nc)',
                      Compatibility='The masked variables are compatible with '+fNEMO_mask,
                      Creator='C. Burgard and P. Mathiot ('+date.today().strftime("%b-%d-%Y")+')')

In [158]:
ds_isfNEMO

<xarray.Dataset>
Dimensions:          (y: 331, x: 360, t: 1)
Coordinates:
    glamt            (y, x) float64 ...
    gphit            (y, x) float64 ...
    ID               int64 133
Dimensions without coordinates: y, x, t
Data variables:
    mask_isf         (y, x, t) float32 nan nan nan nan nan ... nan nan nan nan
    mask_isf_open    (y, x, t) float32 nan nan nan nan nan ... nan nan nan nan
    mask_isf_closed  (y, x, t) float32 nan nan nan nan nan ... nan nan nan nan
    z_min            (y, x, t) float32 nan nan nan nan nan ... nan nan nan nan
    z_max            (y, x, t) float32 nan nan nan nan nan ... nan nan nan nan
    melt_isf_closed  (y, x, t) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    Source:         Based on IMBIE2 (Mask_Iceshelf_IMBIE2_v2.nc)
    Compatibility:  The masked variables are compatible with eORCA1.4.3_OpenS...
    Creator:        C. Burgard and P. Mathiot (Oct-22-2024)

In [157]:
ds_isfNEMO.to_netcdf(inputpath_interim + 'masks_for_eORCA1.nc')